In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None

import xgboost as xgb
root = "C:/Users/HP/Downloads/market"


In [2]:
df = pd.read_pickle(root + 'Finaldata.pkl')
df.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,10.0,9.0,0.900000,1.400000,17.600000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,9.0,8.0,0.888889,3.333333,19.555555,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
3,1,12427,10.0,9.0,0.900000,3.300000,17.600000,10.0,1.0,1.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
4,1,13032,3.0,2.0,0.666667,6.333333,21.666666,10.0,1.0,0.0,...,41.0,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667


In [3]:
def reduce_memory(df):
    
    """
    This function reduce the dataframe memory usage by converting it's type for easier handling.
    
    Parameters: Dataframe
    Return: Dataframe
    """
    
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    
    for col in df.columns:
        if df[col].dtypes in ["int64", "int32", "int16"]:
            
            cmin = df[col].min()
            cmax = df[col].max()
            
            if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            
            elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            
            elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        
        if df[col].dtypes in ["float64", "float32"]:
            
            cmin = df[col].min()
            cmax = df[col].max()
            
            if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            
            elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
    
    print("")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    
    return df

In [4]:
reduce_memory(df)

Memory usage of properties dataframe is : 4428.972461700439  MB

___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  1276.9665126800537  MB
This is  28.83211678832117 % of the initial size


,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,10.0,9.0,0.899902,1.400391,17.593750,10.0,1.0,1.0,...,41.0,0.694824,5.898438,0.706055,6,6,9,0.666504,1.000000,0.666504
1,1,10258,9.0,8.0,0.888672,3.333984,19.562500,10.0,1.0,1.0,...,41.0,0.694824,5.898438,0.706055,6,6,9,0.666504,1.000000,0.666504
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,...,41.0,0.694824,5.898438,0.706055,6,6,9,0.666504,1.000000,0.666504
3,1,12427,10.0,9.0,0.899902,3.300781,17.593750,10.0,1.0,1.0,...,41.0,0.694824,5.898438,0.706055,6,6,9,0.666504,1.000000,0.666504
4,1,13032,3.0,2.0,0.666504,6.332031,21.671875,10.0,1.0,0.0,...,41.0,0.694824,5.898438,0.706055,6,6,9,0.666504,1.000000,0.666504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,206209,43961,3.0,2.0,0.666504,8.000000,23.328125,12.0,1.0,0.0,...,61.0,0.472900,9.921875,0.536621,8,20,9,0.750000,0.700195,0.444336
8474657,206209,44325,1.0,0.0,0.000000,8.000000,9.000000,7.0,0.0,0.0,...,61.0,0.472900,9.921875,0.536621,8,20,9,0.750000,0.700195,0.444336
8474658,206209,48370,1.0,0.0,0.000000,8.000000,30.000000,11.0,0.0,0.0,...,61.0,0.472900,9.921875,0.536621,8,20,9,0.750000,0.700195,0.444336
8474659,206209,48697,1.0,0.0,0.000000,6.000000,9.000000,7.0,0.0,0.0,...,61.0,0.472900,9.921875,0.536621,8,20,9,0.750000,0.700195,0.444336


In [5]:
df['order_diff'] = df.order_number - df.last_ordered_in

In [6]:
top_features=['user_id','product_id','reordered','days_since_prior_order',
 'order_diff',
 'total_product_orders_by_user',
 'order_number',
 'second_time_percent',
 'reorder_propotion_by_user',
 'total_unique_product_by_user',
 'last_ordered_in',
 'reorder_percentage',
 'avg_days_since_last_bought',
 'average_order_size',
 'reorder_in_order',
 'avg_add_to_cart_by_user',
 'add_to_cart_order_mean',
 'unique_users']

In [7]:
df_new=df[top_features]
data=df_new[:200000]
data.head()

,user_id,product_id,reordered,days_since_prior_order,order_diff,total_product_orders_by_user,order_number,second_time_percent,reorder_propotion_by_user,total_unique_product_by_user,last_ordered_in,reorder_percentage,avg_days_since_last_bought,average_order_size,reorder_in_order,avg_add_to_cart_by_user,add_to_cart_order_mean,unique_users
0,1,196,1.0,14.0,1.0,10.0,11.0,0.582520,0.694824,18,10.0,0.776367,17.593750,5.898438,0.706055,1.400391,3.722656,8000
1,1,10258,1.0,14.0,1.0,9.0,11.0,0.552734,0.694824,18,10.0,0.713867,19.562500,5.898438,0.706055,3.333984,4.277344,557
2,1,10326,0.0,14.0,6.0,1.0,11.0,0.521484,0.694824,18,5.0,0.651855,28.000000,5.898438,0.706055,5.000000,4.191406,1923
3,1,12427,0.0,14.0,1.0,10.0,11.0,0.529297,0.694824,18,10.0,0.740723,17.593750,5.898438,0.706055,3.300781,4.761719,1679
4,1,13032,1.0,14.0,1.0,3.0,11.0,0.479736,0.694824,18,10.0,0.657227,21.671875,5.898438,0.706055,6.332031,5.621094,1286


In [8]:
data.columns

Index(['user_id', 'product_id', 'reordered', 'days_since_prior_order',
       'order_diff', 'total_product_orders_by_user', 'order_number',
       'second_time_percent', 'reorder_propotion_by_user',
       'total_unique_product_by_user', 'last_ordered_in', 'reorder_percentage',
       'avg_days_since_last_bought', 'average_order_size', 'reorder_in_order',
       'avg_add_to_cart_by_user', 'add_to_cart_order_mean', 'unique_users'],
      dtype='object')

In [9]:
products = pd.read_csv(root + '/products.csv')

In [10]:
# Merge product names into the main dataset
data = pd.merge(data, products[['product_id', 'product_name']], on='product_id', how='left')
data.head()

,user_id,product_id,reordered,days_since_prior_order,order_diff,total_product_orders_by_user,order_number,second_time_percent,reorder_propotion_by_user,total_unique_product_by_user,last_ordered_in,reorder_percentage,avg_days_since_last_bought,average_order_size,reorder_in_order,avg_add_to_cart_by_user,add_to_cart_order_mean,unique_users,product_name
0,1,196,1.0,14.0,1.0,10.0,11.0,0.582520,0.694824,18,10.0,0.776367,17.593750,5.898438,0.706055,1.400391,3.722656,8000,Soda
1,1,10258,1.0,14.0,1.0,9.0,11.0,0.552734,0.694824,18,10.0,0.713867,19.562500,5.898438,0.706055,3.333984,4.277344,557,Pistachios
2,1,10326,0.0,14.0,6.0,1.0,11.0,0.521484,0.694824,18,5.0,0.651855,28.000000,5.898438,0.706055,5.000000,4.191406,1923,Organic Fuji Apples
3,1,12427,0.0,14.0,1.0,10.0,11.0,0.529297,0.694824,18,10.0,0.740723,17.593750,5.898438,0.706055,3.300781,4.761719,1679,Original Beef Jerky
4,1,13032,1.0,14.0,1.0,3.0,11.0,0.479736,0.694824,18,10.0,0.657227,21.671875,5.898438,0.706055,6.332031,5.621094,1286,Cinnamon Toast Crunch


In [11]:
data.to_csv('data_market.csv',index=False)

In [13]:
import joblib
# Load the model
model = joblib.load('market.pkl')

In [14]:
def get_user_orders(user_id):
    # Filter the dataset for the given user ID
    return data[data['user_id'] == user_id]


def generate_features(user_orders):
    if user_orders.empty:
        return None  # Handle case with no previous orders
    
    # Select relevant features directly without dropping order_number
    relevant_features = user_orders.drop(columns=['user_id', 'product_id', 'product_name','reordered']).iloc[-1:]  # Use the last order as features
    
    return relevant_features  # Return as DataFrame

def predict_reorder(user_features):
    if user_features is None:
        return None, "No previous orders found for this user."
    
    dmatrix = xgb.DMatrix(user_features)
    probabilities = model.predict(dmatrix)
    predictions = [1 if i > 0.5 else 0 for i in probabilities]  # Convert probabilities to binary predictions
    return probabilities, predictions

def suggest_products(user_orders, predictions):
    if not any(predictions):  # If no predictions are 1
        return "No products to suggest based on predictions."
    
    # Filter for previously reordered products only
    reordered_products = user_orders[user_orders['reordered'] == 1]
    
    # Get unique product names that were previously reordered
    suggested_products = reordered_products['product_name'].unique()
    
    if len(suggested_products) == 0:
        return "No previously reordered products to suggest."
    
    return suggested_products  # Return unique product names
    

# Test with a user ID
user_id = 1  # Example user ID
user_orders = get_user_orders(user_id)

# Extract product names as a list
product_names = user_orders['product_name'].tolist()

# Print the product names list
print("Previous Product Names:")
print(product_names)  # This will print only the list of product names

# Generate features from the most recent order
user_features = generate_features(user_orders)

# Make a prediction
probabilities, predictions = predict_reorder(user_features)
print("\nPredicted Reorder Probabilities:")
print(probabilities)
print("\nBinary Predictions:")
print(predictions)

# Suggest future products based on previous orders
suggested_products = suggest_products(user_orders, predictions)
print("\nSuggested Future Products:")
print(suggested_products)

Previous Product Names:
['Soda', 'Pistachios', 'Organic Fuji Apples', 'Original Beef Jerky', 'Cinnamon Toast Crunch', 'Bag of Organic Bananas', 'Organic Unsweetened Vanilla Almond Milk', 'Honeycrisp Apples', 'Organic String Cheese', 'Aged White Cheddar Popcorn', 'XL Pick-A-Size Paper Towel Rolls', 'Creamy Almond Butter', 'Organic Unsweetened Almond Milk', '0% Greek Strained Yogurt', 'Milk Chocolate Almonds', 'Bartlett Pears', 'Zero Calorie Cola', 'Organic Half & Half']

Predicted Reorder Probabilities:
[0.7821769]

Binary Predictions:
[1]

Suggested Future Products:
['Soda' 'Pistachios' 'Cinnamon Toast Crunch' 'Organic String Cheese'
 'Aged White Cheddar Popcorn' 'XL Pick-A-Size Paper Towel Rolls'
 '0% Greek Strained Yogurt' 'Milk Chocolate Almonds' 'Zero Calorie Cola'
 'Organic Half & Half']
